<a href="https://colab.research.google.com/github/harshithamadarapu/Team16_Hinglish-Auto-suggestions/blob/main/DistilBERTTraining_Finetuning%26Evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing necessary libraries
!pip install datasets transformers torch wandb

# importing required modules
from datasets import Dataset
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForMaskedLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
import torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# loading training and validation data from CSV files
train_data = pd.read_csv('/content/cleaned_pre_train.csv')
val_data = pd.read_csv('/content/cleaned_pre_val.csv')

# handling missing values in the phrases column
train_data['phrases'] = train_data['phrases'].fillna('')
val_data['phrases'] = val_data['phrases'].fillna('')

# initializing distilbert tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# tokenizing the data
def tokenize_data(data):
    return data['phrases'].apply(
        lambda x: tokenizer.encode(x, add_special_tokens=True) if isinstance(x, str) else []
    )

# applying the tokenization function
train_tokens = tokenize_data(train_data)
val_tokens = tokenize_data(val_data)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [3]:
# preparing data for BERT model
def prepare_bert_data(tokens):
    input_ids = tokens.tolist()
    attention_masks = [[1] * len(token) for token in input_ids]
    return {'input_ids': input_ids, 'attention_mask': attention_masks}

# converting tokenized data into Dataset objects
def create_dataset(tokens):
    bert_data = prepare_bert_data(tokens)
    return Dataset.from_dict(bert_data)

# creating train and validation datasets
train_dataset = create_dataset(train_tokens)
val_dataset = create_dataset(val_tokens)


In [4]:
# creating data collator for dynamic padding and masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [5]:
# setting up training arguments with gradient accumulation and mixed precision
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,  # reducing batch size
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=True,  # for having mixed precision
    gradient_accumulation_steps=4,  # for accumulating gradients
    logging_steps=100,
    load_best_model_at_end=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# initializing distilbert masked language model
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [7]:
# preparing the trainer with the model, training arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


<ipython-input-7-73026b05ba2a>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# training the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
0,1.518800,3.938618
1,1.310600,3.686892
2,1.280500,3.530458


There were missing keys in the checkpoint model loaded: ['vocab_projector.weight'].


TrainOutput(global_step=17673, training_loss=1.5312499835954851, metrics={'train_runtime': 2300.5496, 'train_samples_per_second': 245.843, 'train_steps_per_second': 7.682, 'total_flos': 3989465275534272.0, 'train_loss': 1.5312499835954851, 'epoch': 2.999830263939574})

In [9]:
# saving the trained model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# loading the saved model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('./results')
model = DistilBertForMaskedLM.from_pretrained('./results')


In [10]:
# defining a function to predict the next words based on the input sentence
def predict_next_words(input_sentence, top_k=5):
    # tokenizing the input sentence
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt')

    # masking the last token for prediction
    masked_input_ids = input_ids.clone()
    masked_input_ids[0, -1] = tokenizer.mask_token_id  # masking the last word

    # creating the attention mask
    attention_mask = (masked_input_ids != tokenizer.pad_token_id).long()

    # getting the model's prediction for the masked token
    with torch.no_grad():
        outputs = model(masked_input_ids, attention_mask=attention_mask)
        predictions = outputs.logits

    # getting the top k predicted tokens for the masked position
    predicted_ids = torch.topk(predictions[0, -1], top_k).indices.tolist()

    # decoding the predicted token ids to words
    predicted_words = tokenizer.decode(predicted_ids, skip_special_tokens=True)

    return predicted_words

# taking sentence input from the user
user_input = input("Enter a sentence: ")

# predicting the next 5 words
predicted_words = predict_next_words(user_input)

# displaying the predicted words
print("Predicted next 5 words:", predicted_words)


Enter a sentence: main tumse pyaar
Predicted next 5 words: hih main ki k


In [11]:
import shutil
shutil.make_archive('/content/model_backup', 'zip', './results')

'/content/model_backup.zip'

In [12]:
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
# to calculate Cosine Similarity function
def compute_cosine_similarity(predicted_sentence, actual_sentence):
    # Converting the  sentences to word vectors (using TF-IDF vectorizer)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([predicted_sentence, actual_sentence])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

In [14]:
# calculating BLEU Score function
def compute_bleu_score(predicted_sentence, actual_sentence):
    # Tokenizing the sentences
    predicted_tokens = predicted_sentence.split()
    actual_tokens = actual_sentence.split()
    return sentence_bleu([actual_tokens], predicted_tokens)

In [15]:
# calculating Jaccard Similarity function
def compute_jaccard_similarity(predicted_sentence, actual_sentence):
    predicted_set = set(predicted_sentence.split())
    actual_set = set(actual_sentence.split())
    intersection = predicted_set.intersection(actual_set)
    union = predicted_set.union(actual_set)
    return len(intersection) / len(union)

In [16]:
def evaluate_model_metrics():
    test_sentences = [
        ("I want to go to the", "I want to go to the park"),
        ("This is a beautiful", "This is a beautiful day"),
        ("Can you please help me with", "Can you please help me with this task")
    ]

    for input_sentence, actual_sentence in test_sentences:
        print(f"Input: {input_sentence}")

        # getting the predicted words using the trained model
        predicted_words = predict_next_words(input_sentence)
        predicted_sentence = input_sentence + " " + predicted_words

        print(f"Predicted: {predicted_sentence}")
        print(f"Actual: {actual_sentence}")

        # computig Cosine Similarity
        cos_sim = compute_cosine_similarity(predicted_sentence, actual_sentence)
        print(f"Cosine Similarity: {cos_sim:.4f}")

        # computing BLEU Score
        bleu = compute_bleu_score(predicted_sentence, actual_sentence)
        print(f"BLEU Score: {bleu:.4f}")

        #computing Jaccard Similarity
        jaccard = compute_jaccard_similarity(predicted_sentence, actual_sentence)
        print(f"Jaccard Similarity: {jaccard:.4f}")

        print("\n" + "-"*50 + "\n")

# Running the evaluation
evaluate_model_metrics()


Input: I want to go to the
Predicted: I want to go to the the it tonight s game
Actual: I want to go to the park
Cosine Similarity: 0.6691
BLEU Score: 0.4617
Jaccard Similarity: 0.5000

--------------------------------------------------

Input: This is a beautiful
Predicted: This is a beautiful one the a people beautiful
Actual: This is a beautiful day
Cosine Similarity: 0.5193
BLEU Score: 0.2985
Jaccard Similarity: 0.5000

--------------------------------------------------

Input: Can you please help me with
Predicted: Can you please help me with you the my please this
Actual: Can you please help me with this task
Cosine Similarity: 0.7297
BLEU Score: 0.4799
Jaccard Similarity: 0.7000

--------------------------------------------------

